In [57]:
#importando as bibliotecas que usarei: Pandas, Docx#
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

In [118]:
class PARTE:

    #busca no BD parâmetro 'parte' (qualquer atributo= nome, CPF, etc)#
    def __init__(self, parte):           
        
        #identifica se PF/PJ (abas diferentes do excel)#
        aba = 'PF' if parte in pd.read_excel('0. Cadastro.xlsm').values else 'PJ'

        #carrega o BD (fornecer nome do arquivo excel) e converte o BD em string#
        self.bd = pd.read_excel('0. Cadastro.xlsm', aba).astype(str)
        
        #cria o dicionário, com todos os atributos da PF/PJ#
        self._criar_dicionário(parte)


    #cria o dicionário (chave= nome do atributo, valor= valor do atributo)#
    def _criar_dicionário(self, parte):
        
        #procura a coluna com 'parte' e retorna o primeiro valor, se não tem retorna None#
        df = self.bd.loc[self.bd.isin([parte]).any(axis=1)]
        
        #cria o dicionário (colunas = chaves, valores = atributos da linha que retornou a 'parte')#       
        self.dicionario = df.iloc[0].to_dict()
        
        #adiciona o atributo endereço ao dicionário#
        self._endereco()

        #adiciona o atributo qualificacao ao dicionário#
        self._qualificacao()
    

    #unifica o endereço em um só atributo#      
    def _endereco(self):
        
        #checa se tem ou não complemento, para não deixar espaço em branco#
        if self.dicionario['xCOMPLEMENTOx'] != 'nan':
            self.dicionario['xNUMERO_COMPLEMENTOx'] = f"{self.dicionario['xNUMEROx']} - {self.dicionario['xCOMPLEMENTOx']}"

        else:
            self.dicionario['xNUMERO_COMPLEMENTOx'] = self.dicionario['xNUMEROx']

        self.dicionario['xENDERECOx'] = (f"{self.dicionario['xLOGRADOUROx']}, nº {self.dicionario['xNUMERO_COMPLEMENTOx']}, Bairro "
            f"{self.dicionario['xBAIRROx']}, CEP {self.dicionario['xCEPx']}, {self.dicionario['xCIDADEx']}/{self.dicionario['xUFx']}")


    #deixa a qualificação pronta e adiciona ao dicionário#  
    def _qualificacao(self):
                
        #diferencia PF/PJ/Condomínio#        
        if 'xCPFx' in self.bd.columns:

            self.dicionario['xQUALIFICAÇÃOx'] = '{}, {}, {}, {}, RG n° {} {}, CPF n° {}, e-mail: {}, com endereço à {}'.format(
                self.dicionario['xNOMEx'], self.dicionario['xNACIONALIDADEx'], self.dicionario['xESTADO_CIVILx'], 
                self.dicionario['xPROFISSAOx'], self.dicionario['xRGx'], self.dicionario['xEXPEDIDOR_RGx'], 
                self.dicionario['xCPFx'], self.dicionario['xEMAILx'],self.dicionario['xENDERECOx'])

        elif self.dicionario['xESCRITORIOx'] == 'GFA':
            self.condominio()

        else:
            self.dicionario['xQUALIFICAÇÃOx'] = ('{}, pessoa jurídica inscrita no CNPJ n° {}, com endereço à {}, devidamente' 
                ' representada neste ato pelo(a) sócio(a) {}').format(self.dicionario['xNOMEx'], self.dicionario['xCNPJx'], 
                self.dicionario['xENDERECOx'],self.dicionario['xADMINISTRADORx'])
    

    #transforma a qualificação do Cliente para síndico e substitui no dicionário#
    def condominio(self):
        self.dicionario['xQUALIFICAÇÃOx'] = (f"{self.dicionario['xNOMEx']}, inscrito no CNPJ n° {self.dicionario['xCNPJx']}, com "
            f"endereço à {self.dicionario['xENDERECOx']}, devidamente representado neste ato pelo seu síndico(a) "
            f"{self.dicionario['xADMINISTRADORx']}")

In [119]:
class PECA: 

    #carrega atributos iniciais da classe PECA (autor, réu, busca = nome da peça)#
    def __init__(self, peca, cliente, adverso = None):

        #Carrega o BD (fornecer nome do arquivo excel e nome da aba da planilha) e converte o BD em string#
        self.bd = pd.read_excel('0. Cadastro.xlsm', 'Peticoes').astype(str)
        
        #cria os atributos e parâmetros da peça a ser feita#
        self._carregar_atributos_parametros(peca)
       
        #cria o dicionário interno da função#
        self._criar_dicionario(cliente, adverso)
      
        #cria o documento especificado em 'peca'#
        self.criar_documento(peca)


    #busca lista de atributos e parâmetros, definidos no BD#
    def _carregar_atributos_parametros(self, peca):

        #define a linha no BD onde tem o parâmetro 'peca'#
        linha = self.bd[self.bd['xPECAx'] == peca].iloc[0]

        #coloca os atributos da peça em maiúsuclo e sem espaço, separados pela ','#
        self.atributos = linha['xATRIBUTOSx'].strip().upper().split(',')

        #coloca os parâmetros da peça em maiúsuclo e sem espaço, separados pela '/'#
        self.parametros = linha['xPARAMETROSx'].strip().split('/')


    #cria o dicionário da classe#   
    def _criar_dicionario(self, cliente, adverso):

        #acrescenta o 'x' na chave do atributo e cria o dicionário#       
        self.dicionario = {'x' + atributo + 'x': parametro for atributo, parametro in zip(self.atributos, self.parametros)}
        
        #executa a função para acrescentar o cliente e adverso ao dicionário#
        self._atualizar_dicionario_com_parte(cliente)
        if adverso:
            self._atualizar_dicionario_com_parte(adverso, 'ADVERSO')  


    #adiciona os atributos de cliente\adverso#
    def _atualizar_dicionario_com_parte(self, parte, prefixo = 'CLIENTE'):                   
        
        #função interna para criar o dicionário com a Classe PARTE#
        def dicionario_parte_unica(parte, prefixo):
            for chave, valor in PARTE(parte).dicionario.items():
                nova_chave = prefixo + chave
                self.dicionario[nova_chave] = valor

        #checa se o número de parte é multiplo(lista) e cria o dicionário#
        if isinstance(parte, list):
            for index, nome in enumerate(parte):
                if index == 0:
                    dicionario_parte_unica(nome, prefixo)                  

                else:
                    self.dicionario['CLIENTExQUALIFICAÇÃOx'] = f"{self.dicionario['CLIENTExQUALIFICAÇÃOx']} e"\
                    f" ({index + 1}) {PARTE(nome).dicionario['xQUALIFICAÇÃOx']}"                  

            self.dicionario['CLIENTExQUALIFICAÇÃOx'] = f"(1) {self.dicionario['CLIENTExQUALIFICAÇÃOx']}"
    
        else:
            dicionario_parte_unica(parte, prefixo)
 

    #cria documento#
    def criar_documento(self, peca):

        #abre o arquivo#    
        documento = Document(f"{self.dicionario['CLIENTExESCRITORIOx']}\{peca}.docx") 

        #função para substituir os textos e salvar o documento#                                 
        self._substituir(documento, self.dicionario, peca)
    

    #substitui os parametros no documento#
    def _substituir(self, documento, dicionario, peca):
       
        #busca no documento por parâmetros e substitui pelos valores do dicionário#
        for paragrafo in documento.paragraphs:
            for chave in dicionario:
                if chave in paragrafo.text:
                    paragrafo.text = paragrafo.text.replace(chave, dicionario[chave])
        
        #duas opções de salvamento de nome do arquivo, uma que será usada apenas na classe herdada
        try:
            documento.save(f"Documentos Prontos\{peca} - {self.dicionario['CLIENTExNOMEx']} x {self.dicionario['ADVERSOxNOMEx']}.docx")
        except:
            documento.save(f"Documentos Prontos\{peca} - {self.dicionario['CLIENTExNOMEx']}.docx")

In [121]:
lista = ['Fabiano', 'Sandra',]
cliente = 'Residencial'

x = PECA('Procuração', cliente)
x = PECA('Procuração', lista)

print(type(lista))

<class 'list'>
